# Updating spaCy's Named Entity Recognition System

Pretrained models are simple to use, but they're unlikely to obtain state-of-the-art performance if your data differs even slightly from the type of data it was trained on. If state-of-the-art performance is what you're looking for, at some point you're going to want to train your own model. Luckily, spaCy allows this, too. In fact, spaCy offers us two options: it allows us to train a model from scratch, or to continue training its pretrained model with our own data.

In [1]:
from IPython.display import HTML, display
import tabulate
import spacy

nlp = spacy.load("en")
text = "Theresa May is a British politician serving as Prime Minister of the United Kingdom and Leader of the Conservative Party since 2016. "

doc = nlp(text)
entities = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]
display(HTML(tabulate.tabulate(entities, tablefmt='html')))

Theresa,B,ORG
May,B,DATE
is,O,
a,O,
British,B,NORP
politician,O,
serving,O,
as,O,
Prime,O,
Minister,O,
of,O,


In [2]:
training_texts = [
    (["Theresa", "May", "is", "determined", "to", "leave", "the", "EU", "in", "March", "."],
     ["B-PERSON", "L-PERSON", "O", "O", "O", "O", "O", "U-ORG", "O", "U-DATE", "O"]
    ),
    (["Theresa", "May", "says", "she", "will", "seek", "a", "pragmatic", "Brexit", "deal", "."],
     ["B-PERSON", "L-PERSON", "O", "O", "O", "O", "O", "O", "O", "O", "O"]
    ),
    (["Theresa", "May", "vows", "to", "battle", "in", "Brussels", "."],
     ["B-PERSON", "L-PERSON", "O", "O", "O", "O", "U-GPE", "O"]
    )
]


In [3]:
from spacy.tokens import Doc
from spacy.gold import GoldParse

training_data = []
for tokens, annotation in training_texts:
    doc = Doc(nlp.vocab, words=tokens)
    gold = GoldParse(doc, entities=annotation)
    training_data.append((doc, gold))

In [4]:
import random
from tqdm import tqdm_notebook as tqdm

for _ in tqdm(range(10)):
    random.shuffle(training_data)
    for doc, gold in training_data:
        nlp.update([doc], [gold], drop=0.3)

Let's now test the model on the same sentence as before. The output shows it still recognizes all the correct entities it found before, but now it has also identified `Theresa May` as a person. Hurray!

In [5]:
text = "Theresa May is a British politician serving as Prime Minister of the United Kingdom and Leader of the Conservative Party since 2016. "

doc = nlp(text)
entities = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]
display(HTML(tabulate.tabulate(entities, tablefmt='html')))

Theresa,B,PERSON
May,I,PERSON
is,O,
a,O,
British,B,NORP
politician,O,
serving,O,
as,O,
Prime,O,
Minister,O,
of,O,


## Training an NER model on Dutch CONLL data

In [6]:
!wget https://raw.githubusercontent.com/teropa/nlp/master/resources/corpora/conll2002/ned.train -P data/ner/
!wget https://raw.githubusercontent.com/teropa/nlp/master/resources/corpora/conll2002/ned.testa -P data/ner/
!wget https://raw.githubusercontent.com/teropa/nlp/master/resources/corpora/conll2002/ned.testb -P data/ner/

--2019-02-04 20:11:19--  https://raw.githubusercontent.com/teropa/nlp/master/resources/corpora/conll2002/ned.train
Resolving raw.githubusercontent.com... 151.101.36.133
Connecting to raw.githubusercontent.com|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2377174 (2.3M) [text/plain]
Saving to: 'data/ner/ned.train.6'

data/ner/ned.train. 100%[=====================>]   2.27M  3.88MB/s   in 0.6s   

2019-02-04 20:11:21 (3.88 MB/s) - 'data/ner/ned.train.6' saved [2377174/2377174]

--2019-02-04 20:11:21--  https://raw.githubusercontent.com/teropa/nlp/master/resources/corpora/conll2002/ned.testa
Resolving raw.githubusercontent.com... 151.101.36.133
Connecting to raw.githubusercontent.com|151.101.36.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 450785 (440K) [text/plain]
Saving to: 'data/ner/ned.testa.6'

data/ner/ned.testa. 100%[=====================>] 440.22K  --.-KB/s   in 0.1s   

2019-02-04 20:11:21 (3.41 MB/s) - '

In [8]:
from operator import itemgetter

train_file = "data/ner/ned.train"
dev_file = "data/ner/ned.testa"
test_file = "data/ner/ned.testb"

def read_conll_file(f):
    data = []
    with open(f) as i:
        sentences = i.read().strip().split("\n\n")
        
    for sentence in sentences:
        data.append([token.split() for token in sentence.split("\n")])

    return data
        
train_data = read_conll_file(train_file)
dev_data = read_conll_file(dev_file)
test_data = read_conll_file(test_file)

In [9]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

def evaluate(model, data, verbose=0): 

    ner = model.get_pipe("ner")
    
    correct, predicted = [], []
    for sentence in data:
        tokens = [t[0] for t in sentence]
        ent_labels = [t[2].split("-")[-1] for t in sentence]
        
        doc = Doc(model.vocab, words=tokens)
        ner(doc)
        
        pred_labels = [t.ent_type_ or "O" for t in doc]
        correct += ent_labels
        predicted += pred_labels
        
    if verbose:
        print(classification_report(correct, predicted))
    
    return precision_recall_fscore_support(correct, predicted, average="micro")

In [10]:
nlp = spacy.load("nl")
evaluate(nlp, test_data, verbose=1)

5195
              precision    recall  f1-score   support

         LOC       0.50      0.85      0.63       823
        MISC       0.66      0.45      0.54      1597
           O       0.99      0.99      0.99     63236
         ORG       0.71      0.64      0.68      1433
         PER       0.77      0.81      0.79      1905

   micro avg       0.97      0.97      0.97     68994
   macro avg       0.73      0.75      0.72     68994
weighted avg       0.97      0.97      0.97     68994



(0.9663159115285387, 0.9663159115285387, 0.9663159115285387, None)

In [11]:
from spacy.gold import iob_to_biluo

training_data = []
for sentence in train_data:
    tokens = [t[0] for t in sentence]
    ent_labels = iob_to_biluo([t[2] for t in sentence])
    doc = Doc(nlp.vocab, words=tokens)
    gold = GoldParse(doc, entities=ent_labels)
    training_data.append((doc, gold))

In [12]:
from spacy.util import minibatch
from pathlib import Path

def train(train_docs, dev_data, output_dir, model=None, max_epochs=100): 
    
    if not model: 
        model = spacy.blank("nl")
        ner = model.create_pipe("ner")
        model.add_pipe(ner, last=True)
        for label in ["PER", "LOC", "ORG", "MISC"]: 
            ner.add_label(label)
        model.begin_training()
        
    other_pipes = [pipe for pipe in model.pipe_names if pipe != 'ner']
    fscore_history = []
    patience=3
        
    with model.disable_pipes(*other_pipes):
    
        for i in range(max_epochs):
            print("Epoch", i)
            losses = {}
            random.shuffle(train_docs)
            batches = minibatch(train_docs, size=32)
            for batch in tqdm(batches):
                docs, golds = zip(*batch)
                model.update(
                    docs,
                    golds,
                    drop=0.4,
                    losses=losses)
            print("Training Loss:", losses)
            
            _, _, dev_f, _ = evaluate(model, dev_data)
            print("Development F-score:", dev_f)
            
            if len(fscore_history) > 0 and dev_f > max(fscore_history): 
                if output_dir is not None:
                    output_dir = Path(output_dir)
                    if not output_dir.exists():
                        output_dir.mkdir()
                    model.to_disk(output_dir)
                    print("Saved model to", output_dir)
            
            fscore_history.append(dev_f)
            
            if max(fscore_history) > max(fscore_history[-patience:]):
                print("No improvement on development set. Stop training.")
                break

In [13]:
output_dir_scratch = "models/spacy_ner_scratch"
train(training_data, dev_data, model=None, output_dir=output_dir_scratch)

Epoch 0



Training Loss: {'ner': 17.634281397026918}
Development F-score: 0.955536135165912
Epoch 1



Training Loss: {'ner': 8.124413278556299}
Development F-score: 0.9652551574375678
Saved model to models/spacy_ner_scratch
Epoch 2



Training Loss: {'ner': 5.07255810720055}
Development F-score: 0.9674531924472339
Saved model to models/spacy_ner_scratch
Epoch 3



Training Loss: {'ner': 3.594017944427045}
Development F-score: 0.9692539922141894
Saved model to models/spacy_ner_scratch
Epoch 4



Training Loss: {'ner': 2.818861507985981}
Development F-score: 0.9607266756706655
Epoch 5



Training Loss: {'ner': 2.3161421975320895}
Development F-score: 0.9678769100394587
Epoch 6



Training Loss: {'ner': 1.934440403581468}
Development F-score: 0.9677974629909165
No improvement on development set. Stop training.


In [14]:
output_dir_cntd = "models/spacy_ner_cntd"
train(training_data, dev_data, model=nlp, output_dir=output_dir_cntd)

Epoch 0



Training Loss: {'ner': 6.690205245732558}
Development F-score: 0.9751860385053361
Epoch 1



Training Loss: {'ner': 3.8389393237998024}
Development F-score: 0.9784698498450782
Saved model to models/spacy_ner_cntd
Epoch 2



Training Loss: {'ner': 2.788690348488808}
Development F-score: 0.9790259791848733
Saved model to models/spacy_ner_cntd
Epoch 3



Training Loss: {'ner': 2.0977905896412414}
Development F-score: 0.9789200497868171
Epoch 4



Training Loss: {'ner': 1.7780554252080347}
Development F-score: 0.9781520616509096
Epoch 5



Training Loss: {'ner': 1.4171092849177291}
Development F-score: 0.9782579910489658
No improvement on development set. Stop training.


In [17]:
nlp_base = spacy.load("nl")
nlp_scratch = spacy.load(output_dir_scratch)
nlp_cntd = spacy.load(output_dir_cntd)

print("\n********** Base Model **********")
evaluate(nlp_base, test_data, verbose=1)
print("\n********** New Model **********")
evaluate(nlp_scratch, test_data, verbose=1)
print("\n********** Continued Model **********")
evaluate(nlp_cntd, test_data, verbose=1)



********** Base Model **********
              precision    recall  f1-score   support

         LOC       0.50      0.85      0.63       823
        MISC       0.66      0.45      0.54      1597
           O       0.99      0.99      0.99     63236
         ORG       0.71      0.64      0.68      1433
         PER       0.77      0.81      0.79      1905

   micro avg       0.97      0.97      0.97     68994
   macro avg       0.73      0.75      0.72     68994
weighted avg       0.97      0.97      0.97     68994


********** New Model **********
              precision    recall  f1-score   support

         LOC       0.76      0.78      0.77       823
        MISC       0.78      0.59      0.67      1597
           O       1.00      1.00      1.00     63236
         ORG       0.71      0.67      0.69      1433
         PER       0.76      0.89      0.82      1905

   micro avg       0.98      0.98      0.98     68994
   macro avg       0.80      0.79      0.79     68994
weighted a

(0.9825926892193524, 0.9825926892193524, 0.9825926892193524, None)

## Conclusion
Unfortunately, the continued training of a pretrained model is not without its challenges. Most importantly, you need to make sure that your model doesn't overfit on the new training data and loses its ability to label the type of data it was originally trained on. A related challenge is that of [catastrophic forgetting](https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting), which typically occurs when weights are shared between several NLP tasks. Still, when you get it right, finetuning an existing model is a powerful way of training high-quality model with a limited amount of data.